In [1]:
import pandas as pd
import numpy as np

### Merge the marks and labels file

In [2]:
df_nas = pd.read_csv("../../../data/nas-20180226T131253Z-001/nas/nas-pupil-marks.csv")
df_nasLabls = pd.read_csv("../../../data/nas-20180226T131253Z-001/nas/nas-labels.csv")

In [3]:
df_nas.shape, df_nasLabls.shape

((185348, 64), (241, 4))

In [4]:
# Get cols with nan
df_nas.columns[df_nas.isnull().any()]

Index(['Use computer', 'Maths %', 'Reading %', 'Science %', 'Social %'], dtype='object')

In [5]:
# Count of the nan's
#df_nas.isnull().sum()
null_cols = df_nas.columns[df_nas.isnull().any()]
df_nas[null_cols].isnull().sum()

Use computer    19162
Maths %         92667
Reading %       92077
Science %       94356
Social %        95777
dtype: int64

In [6]:
# Check for Duplicates
df_nas.duplicated().sum()

16

In [7]:
df_nas.duplicated(subset=['STUID']).sum()

339

In [8]:
df_nas.duplicated(subset=['STUID', 'District']).sum()

339

In [9]:
# Remove the duplicates
#df_nas = df_nas.drop_duplicates(subset=['STUID', 'District'], keep='first')
df_nas = df_nas.drop_duplicates(subset=['STUID'], keep='first')
df_nas.shape

(185009, 64)

In [10]:
df_nas.duplicated(subset=['STUID']).sum()

0

In [11]:
df_nas.duplicated().sum()

0

In [12]:
df_nasLabls.dtypes

Column    object
Name      object
Level     object
Rename    object
dtype: object

In [13]:
# For merging we need to update the cols to str 
# Change all to string and replace nan to NAN - str changes NAN to nan

df_nas = df_nas.applymap(str).replace('nan',np.nan)

In [14]:
df_nas.dtypes.value_counts()

object    64
dtype: int64

In [15]:
# Function to merge the df's - to get the lable values
def df_merge(df,dfLbls):
    df_cols = df.columns.values
    df_Lbls_cols = dfLbls['Column'].unique() #dfLbls.columns.values
    for col in df_cols:
        if col in df_Lbls_cols:
            #print("processing -",col) 
            t = dfLbls.loc[dfLbls['Column'] == col,['Level','Rename']]
            t.columns = [col, 'Rename']
            #mappy = dict(zip(t.Level,t.Rename))
            #df = pd.merge(df,t,on = 'Age',how='left')
            df = pd.merge(df,t,on = col,how='left')
            
            # Dont do this...as can give wrong summation  results...need the NAN's
            #df['Rename'].fillna(df[col], inplace=True)
            
            df.drop(col, axis=1, inplace=True)
            df.rename(columns={'Rename': col}, inplace=True)
        else:
           print("Column does not exist-",col) 
    return(df)

In [16]:
%%time
df_nas = df_merge(df_nas,df_nasLabls)

Column does not exist- STUID
Column does not exist- District
Column does not exist- Use computer
Column does not exist- Maths %
Column does not exist- Reading %
Column does not exist- Science %
Column does not exist- Social %
Wall time: 1min 59s


In [17]:
df_nas.shape

(355043, 64)

In [18]:
df_nas.duplicated().sum()

170034

In [19]:
df_nas.duplicated(subset=['STUID']).sum()

170034

In [20]:
df_nas = df_nas.drop_duplicates(subset=['STUID'], keep='first')
df_nas.shape

(185009, 64)

In [21]:
df_nas.to_csv("nas_2014.csv", index=False)

### Get the difference between the min and max within the factors

In [22]:
df_nas.dtypes.value_counts()

object    64
dtype: int64

In [23]:
df_nas[['Maths %','Science %','Social %','Reading %']] = df_nas[['Maths %','Science %','Social %','Reading %']].apply(pd.to_numeric)

In [24]:
# Function to get the marks difference between the min and max in a factor
def get_marks_diff2(dfnas):
    
    # Get all the columns to get the diff - the factors
    #factors = df_nas.drop(['STUID','District'],axis=1).columns.values
    #factors = df_nas.columns.values
    
    factors = ['Gender', 'Age','Siblings','Handicap', 'Father edu', 'Mother edu', 'Father occupation',
               'Mother occupation', 'Below poverty', 'Use calculator','Use Internet', 'Use dictionary',
               'Computer use', 'Library use','# Books','Distance','Private tuition','Historical excursions',
               'Watch TV', 'Read magazine','Play games', 'Help in household']
    
    # Create a empty df
    df_nas8 = pd.DataFrame(columns =['Factor','Total %', 'Maths %', 'Science %', 'Social %', 'Reading %'])
    
    # Process For all factors
    for col in factors:
        df = dfnas.groupby(col)['Maths %','Science %','Social %','Reading %'].mean()
        df['Total %'] = df[['Maths %', 'Reading %', 'Science %', 'Social %']].mean(axis=1)
    
        # Calculate the diff
        tot = df['Total %'].max() - df['Total %'].min()
        mth = df['Maths %'].max() - df['Maths %'].min()
        sci = df['Science %'].max() - df['Science %'].min()
        soc = df['Social %'].max() - df['Social %'].min()
        rdg = df['Reading %'].max() - df['Reading %'].min()
    
        # Append the row
        df_nas8 = df_nas8.append(pd.Series([col,tot,mth,sci,soc,rdg], index=df_nas8.columns ), ignore_index=True)
    
    return(df_nas8)

In [25]:
df = get_marks_diff2(df_nas)

In [26]:
df.set_index('Factor', inplace=True)

In [27]:
df.to_csv("nas_byFactor_2014.csv")